In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

# Read data from file

path_to_train_file = "./cell2celltrain.csv"
path_to_test_file = "./cell2cellholdout.csv"
train_data = pd.read_csv(path_to_train_file, encoding="utf-8")
test_data = pd.read_csv(path_to_test_file, encoding="utf-8")
print(train_data.shape)
print(test_data.shape)

FileNotFoundError: [Errno 2] File b'./cell2celltrain.csv' does not exist: b'./cell2celltrain.csv'

In [ ]:
print("*********Train_set***********")
print(train_data.describe())
print("\n")
print("*********Test_set************")
print(test_data.describe())

*********Train_set***********
         CustomerID  MonthlyRevenue  MonthlyMinutes  TotalRecurringCharge  \
count  5.104700e+04    50891.000000    50891.000000          50891.000000   
mean   3.201957e+06       58.834492      525.653416             46.830088   
std    1.169056e+05       44.507336      529.871063             23.848871   
min    3.000002e+06       -6.170000        0.000000            -11.000000   
25%    3.100632e+06       33.610000      158.000000             30.000000   
50%    3.201534e+06       48.460000      366.000000             45.000000   
75%    3.305376e+06       71.065000      723.000000             60.000000   
max    3.399994e+06     1223.380000     7359.000000            400.000000   

       DirectorAssistedCalls  OverageMinutes  RoamingCalls  PercChangeMinutes  \
count           50891.000000    50891.000000  50891.000000       50680.000000   
mean                0.895229       40.027785      1.236244         -11.547908   
std                 2.228546     

In [3]:
train_data = train_data.dropna().drop_duplicates()
print(train_data.shape)

(49752, 58)


In [7]:
# First approach - discard all non numeric values from dataset
train_data_feature = train_data[["Churn"]]
train_data_num = train_data.select_dtypes(include=['float64', 'int64'])
train_data_cat = train_data.select_dtypes(include=['object'])
train_data_num = train_data_num.drop(["CustomerID"], axis=1)
print(train_data_num.shape)
print(train_data_cat.shape)
print(train_data_feature.shape)

(49752, 34)
(49752, 23)
(49752, 1)


In [15]:
# Scalling train data

train_data_num = (train_data_num - train_data_num.mean()) / train_data_num.std()
train_data_num.head()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,Handsets,HandsetModels,CurrentEquipmentDays,AgeHH1,AgeHH2,RetentionCalls,RetentionOffersAccepted,ReferralsMadeBySubscriber,IncomeGroup,AdjustmentsToCreditRating
0,-0.783383,-0.579027,-1.045265,-0.288503,-0.41371,-0.124521,-0.568064,-0.452478,-0.587668,-0.311708,...,0.146455,0.486936,-0.075393,1.387937,-0.883853,4.775626,-0.128319,-0.19316,-0.129457,-0.140292
1,-0.941561,-0.974478,-1.255707,-0.400877,-0.41371,-0.124521,0.029924,0.029852,-0.631933,-0.376305,...,0.146455,-0.616505,4.455107,0.392083,0.871516,-0.175754,-0.128319,-0.19316,0.192034,-0.140292
2,-0.467479,-0.978262,-0.371850,-0.400877,-0.41371,-0.124521,0.037740,0.029852,-0.665131,-0.376305,...,-0.604954,-0.616505,5.675924,-0.241642,0.202804,-0.175754,-0.128319,-0.19316,0.513526,-0.140292
3,0.531681,1.489051,1.185423,0.156495,-0.41371,-0.124521,0.659178,0.235477,5.089212,0.334259,...,5.406313,2.693818,0.309085,-0.060578,-0.883853,-0.175754,-0.128319,-0.19316,0.513526,-0.140292
4,-0.938176,-0.993399,-1.255707,-0.400877,-0.41371,-0.124521,0.045557,0.024775,-0.665131,-0.376305,...,1.649271,1.590377,1.870780,0.663680,1.373050,-0.175754,-0.128319,-0.19316,1.478002,2.472881


In [11]:
train_data_cat = pd.get_dummies(train_data_cat)
train_data_cat.head()

,Churn_No,Churn_Yes,ServiceArea_AIRAIK803,ServiceArea_AIRAND864,ServiceArea_AIRASH828,ServiceArea_AIRAUG706,ServiceArea_AIRBEA843,ServiceArea_AIRCAM803,ServiceArea_AIRCHA843,ServiceArea_AIRCOL803,...,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student,MaritalStatus_No,MaritalStatus_Unknown,MaritalStatus_Yes
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [14]:
train_data = pd.concat([train_data_num, train_data_cat], axis = 1)
train_data.head()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,Occupation_Crafts,Occupation_Homemaker,Occupation_Other,Occupation_Professional,Occupation_Retired,Occupation_Self,Occupation_Student,MaritalStatus_No,MaritalStatus_Unknown,MaritalStatus_Yes
0,-0.783383,-0.579027,-1.045265,-0.288503,-0.41371,-0.124521,-0.568064,-0.452478,-0.587668,-0.311708,...,0,0,0,1,0,0,0,1,0,0
1,-0.941561,-0.974478,-1.255707,-0.400877,-0.41371,-0.124521,0.029924,0.029852,-0.631933,-0.376305,...,0,0,0,1,0,0,0,0,0,1
2,-0.467479,-0.978262,-0.371850,-0.400877,-0.41371,-0.124521,0.037740,0.029852,-0.665131,-0.376305,...,1,0,0,0,0,0,0,0,0,1
3,0.531681,1.489051,1.185423,0.156495,-0.41371,-0.124521,0.659178,0.235477,5.089212,0.334259,...,0,0,1,0,0,0,0,1,0,0
4,-0.938176,-0.993399,-1.255707,-0.400877,-0.41371,-0.124521,0.045557,0.024775,-0.665131,-0.376305,...,0,0,0,1,0,0,0,0,0,1


In [16]:
# Split data into cluster column and all others
X = np.array(train_data.astype(float))
# preprocessing for feature column
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
Y = lb.fit_transform(train_data_feature)
print(Y[: 10])

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
kmeans.fit(X)

correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == Y[i]:
        correct += 1

print(correct/len(X))

[[1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]
0.3760049847242322


In [ ]:
# That's pretty bad results. Model needs some tuning.

kmeans = KMeans(n_clusters=2, max_iter=600, algorithm="auto")
kmeans.fit(X)

In [ ]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == Y[i]:
        correct += 1

print(correct/len(X))

In [ ]:
# Data needs some scalling
from sklearn.preprocessing import MinMaxScaler
scaller = MinMaxScaler()
X_scaled = scaller.fit_transform(X)
kmeans.fit(X)
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == Y[i]:
        correct += 1

print(correct/len(X))

In [ ]:
train_data.head

In [ ]:
from sklearn.cluster import 
scaller = MinMaxScaler()
X_scaled = scaller.fit_transform(X)

kmeans.fit(X)
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == Y[i]:
        correct += 1

print(correct/len(X))